In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 20000
maxlen = 100


train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train = train.sample(frac=1)

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values
print(y.shape)

Using TensorFlow backend.


(95851, 6)


In [2]:
print('test len',len(test))

test len 226998


In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

In [15]:
def get_cnn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Conv1D(64,
             3,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [16]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_cnn_model()
        batch_size = 64
        epochs = 10
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_weights_only=True,
                                     save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model.load_weights(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        train_pred[test_index] = model.predict(hold_out_x)
    test_pred = test_pred / fold_cnt
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/cnn_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/cnn_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

Train on 63900 samples, validate on 31951 samples
Epoch 1/10
63900/63900 [==============================] - 10s 163us/step - loss: 0.0915 - acc: 0.9736 - val_loss: 0.0535 - val_acc: 0.9810
Epoch 2/10
63900/63900 [==============================] - 9s 147us/step - loss: 0.0499 - acc: 0.9815 - val_loss: 0.0519 - val_acc: 0.9815
Epoch 3/10
63900/63900 [==============================] - 9s 148us/step - loss: 0.0413 - acc: 0.9843 - val_loss: 0.0547 - val_acc: 0.9813
Epoch 4/10
63900/63900 [==============================] - 9s 148us/step - loss: 0.0345 - acc: 0.9865 - val_loss: 0.0585 - val_acc: 0.9810
Epoch 5/10
63900/63900 [==============================] - 9s 148us/step - loss: 0.0296 - acc: 0.9882 - val_loss: 0.0623 - val_acc: 0.9813
Epoch 6/10
63900/63900 [==============================] - 9s 148us/step - loss: 0.0252 - acc: 0.9899 - val_loss: 0.0709 - val_acc: 0.9810
Epoch 7/10
63900/63900 [==============================] - 9s 147us/step - loss: 0.0213 - acc: 0.9915 - val_loss: 0.0769 -

(95851, 6) (226998, 6)
done


In [10]:
def get_nn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [12]:
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_nn_model()
        batch_size = 64
        epochs = 10
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_weights_only=True,
                                     save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model.load_weights(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        train_pred[test_index] = model.predict(hold_out_x)
    test_pred = test_pred / fold_cnt
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)

Train on 63900 samples, validate on 31951 samples
Epoch 1/10
63900/63900 [==============================] - 9s 137us/step - loss: 0.1356 - acc: 0.9633 - val_loss: 0.0774 - val_acc: 0.9741
Epoch 2/10
63900/63900 [==============================] - 9s 138us/step - loss: 0.0649 - acc: 0.9780 - val_loss: 0.0556 - val_acc: 0.9805
Epoch 3/10
63900/63900 [==============================] - 9s 134us/step - loss: 0.0515 - acc: 0.9813 - val_loss: 0.0519 - val_acc: 0.9814
Epoch 4/10
63900/63900 [==============================] - 8s 131us/step - loss: 0.0455 - acc: 0.9830 - val_loss: 0.0518 - val_acc: 0.9815
Epoch 5/10
63900/63900 [==============================] - 8s 128us/step - loss: 0.0413 - acc: 0.9840 - val_loss: 0.0515 - val_acc: 0.9815
Epoch 6/10
63900/63900 [==============================] - 8s 129us/step - loss: 0.0370 - acc: 0.9856 - val_loss: 0.0516 - val_acc: 0.9818
Epoch 7/10
63900/63900 [==============================] - 8s 132us/step - loss: 0.0334 - acc: 0.9871 - val_loss: 0.0547 - 

63901/63901 [==============================] - 8s 127us/step - loss: 0.0268 - acc: 0.9896 - val_loss: 0.0619 - val_acc: 0.9806
Epoch 10/10
63901/63901 [==============================] - 8s 129us/step - loss: 0.0245 - acc: 0.9906 - val_loss: 0.0631 - val_acc: 0.9802
(95851, 6) (226998, 6)


In [14]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/nn_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/nn_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')